In [1]:
import websocket
import time
import threading
import requests
import json
import pandas as pd
import numpy as np
import sqlite3
from datetime import datetime, timedelta
import hashlib
import hmac
import math
import sys
import logging
import os
import getopt
import traceback

# pd.set_option('display.height', 1000)
# pd.set_option('display.max_rows', 500)		# 显示行数
pd.set_option('display.max_columns', 500)	# 显示列数
pd.set_option('display.width', 1000)		# 显示宽度

# worthDir = '../pkl'  # '/home/at/ipynb/pd' ,  '/home/at/py/ipynb/'
# dbfile = '%s/%s' % (worthDir, 'binance.db') # '/mnt/c/milu/docx/strategy/v2.1/binance' ,  '/mnt/e/milu/py'

worthDir = '.'  # '/home/at/ipynb/pd' ,  '/home/at/py/ipynb/'
dbfile = '%s/%s' % (worthDir, 'worth.db') # '/mnt/c/milu/docx/strategy/v2.1/binance' ,  '/mnt/e/milu/py'

serverTimeadj = 10
locTimeadj = 0 # 28800
newClientOrderId = ''
orderId = ''
name = 'similarity'
curDate = time.strftime("%Y%m%d")
timestamp = int(round((time.time() ) * 1000) )

serverUrl = 'http://testnet.binancefuture.com'
# serverUrl = 'http://www.binance.com'
#ation126@hotmail.com
# api_key    = "fcc2838327a124367acd634323b93b1fb53d6fc66e84d679169a78adcaf1bf3e"  # 密钥
# secret_key = "4f58b518cecdfe9c574cd2aa9cbbea429796673d6226424bdda1f0155cd78876"  # 私钥

#binance@snapmail.cc Test2022
api_key     = "d7188e86617bec4e95daa012fbaabf4a84ce1b12992019b7125df19eb536ad42"  # 密钥
secret_key  = "cb68c9cf4303fc4cd11da655df3a978d41fcbf3b1e927f6e6d49ef41388a6749"  # 私钥



recvWindow = '5000'

url = f'{serverUrl}/fapi/v1/time'
res = requests.get(url)
rt = json.loads(res.text)

if 'code' in rt and 'msg' in rt:
    logger.error(f'res : {res.text}')
else:
    serverTime = rt['serverTime']
    serverTimeadj = ((rt['serverTime']- timestamp) ) //1000

df_balance = pd.DataFrame()
df_account = pd.DataFrame()
df_position = pd.DataFrame()
df_order = pd.DataFrame()
df_trade = pd.DataFrame()

version = "1.0.1"
print(sys.version)
# print(sys.executable)

print( f'{time.strftime("%Y-%m-%d %H:%M:%S",time.localtime(time.time()+locTimeadj))} *****import**v{version}*** ')

3.8.2 (default, Jul 16 2020, 14:00:26) 
[GCC 9.3.0]
2022-10-01 08:52:28 *****import**v1.0.1*** 


In [2]:
###日志 logger##20220825A######
import logging
import sys
import time

formatter = logging.Formatter('%(asctime)s:%(levelname)s:%(name)s:%(filename)s:%(lineno)d:%(funcName)s:%(process)s: %(message)s')

def make_logger(name, log_level=logging.INFO, log_file="log.txt", file_mode="a"): #w写 a追加
    logger = logging.getLogger(name)
    logger.setLevel(level=log_level)
    handler = logging.FileHandler(log_file, mode=file_mode)
    handler.setLevel(level=log_level)
    handler.setFormatter(formatter)
    logger.addHandler(handler)
    handler = logging.StreamHandler(sys.stdout)
    handler.setLevel(level=log_level)
    handler.setFormatter(formatter)
    logger.addHandler(handler)
    return logger


logger = make_logger(name, log_level=logging.DEBUG, log_file= "../logs/%s_%sw.log" % (name if ('name' in dir() ) else 'similarity', curDate if ('curDate' in dir() ) else time.strftime("%Y%m%d") ) )

print( f'{time.strftime("%Y-%m-%d %H:%M:%S",time.localtime(time.time()+locTimeadj))} *****logging***** ')

2022-10-01 08:52:30 *****logging***** 


In [13]:
###查询账户余额 balance###QryBalance###
def QryBalance():

    global df_balance
    global serverUrl, api_key, secret_key, recvWindow

    timestamp = int(round((time.time() +serverTimeadj) * 1000) )  # 时间戳ms
    url = f'{serverUrl}/fapi/v2/balance'

    query_string = 'recvWindow=%s&timestamp=%s' % (recvWindow,str(timestamp))
    signature = hmac.new(secret_key.encode('utf-8'), query_string.encode('utf-8'), hashlib.sha256).hexdigest() # 用secret_key时间戳hmac sha256加密
    params = {"recvWindow":recvWindow, "timestamp":timestamp, "signature":signature}
    headers = {"X-MBX-APIKEY":api_key}

    res = requests.get(url, headers=headers, params=params)
    rt = json.loads(res.text)

    print(f"{rt = }")

    if 'code' in rt and 'msg' in rt:
        logger.error(f'res: {res.text}')
    else :
        rt_df = pd.DataFrame([rt[1] ])
        # rt_df['systemTime'] = rt_df['updateTime'].apply(lambda x: datetime.fromtimestamp(x/1000)) 
        rt_df['updateDate'] = pd.to_datetime(rt_df['updateTime'], unit="ms", origin="1970-01-01 08:00:00")
        rt_df['queryTime'] = pd.to_datetime(int(time.time()), unit='s', origin="1970-01-01 08:00:00")

        rt_df['availableBalance']  = pd.to_numeric(rt_df['availableBalance'] , errors='coerce', downcast='float')
        rt_df['balance']  = pd.to_numeric(rt_df['balance'] , errors='coerce', downcast='float')
        rt_df['crossWalletBalance']  = pd.to_numeric(rt_df['crossWalletBalance'] , errors='coerce', downcast='float')
        rt_df['crossUnPnl']  = pd.to_numeric(rt_df['crossUnPnl'] , errors='coerce', downcast='float')
        rt_df['maxWithdrawAmount']  = pd.to_numeric(rt_df['maxWithdrawAmount'] , errors='coerce', downcast='float')
        rt_df['access'] = 'similarity'

        rt_df.sort_index(axis=1, ascending=True, inplace=True)

        df_balance = pd.concat([df_balance, rt_df ], ignore_index=True)
        df_b = df_balance[['asset','balance','availableBalance','updateTime','queryTime', ] ]

        conn = sqlite3.connect(dbfile)
        # df_balance = pd.read_sql(' select * from balance ', conn) 
        rt_df.to_sql('balance', con=conn, if_exists='append', index=False) #,
        conn.close()

        df_balance.to_pickle('%s/%s.pkl' % (worthDir, "balancedf" ) )
        # df_bal = pd.read_pickle('%s/%s.pkl' % (worthDir, "balancedf" ) )

        logger.info(f'df_b :\n{df_b}')

print( f'{time.strftime("%Y-%m-%d %H:%M:%S",time.localtime(time.time()+locTimeadj))} *****查询账户余额 balance###QryBalance**v{version}*** ')

2022-09-30 22:40:14 *****查询账户余额 balance###QryBalance**v1.0.1*** 


In [9]:
# QryBalance()


log_file= "../logs/%s_%sw.log" % (g_name if ('g_name' in dir() ) else 'similarity', g_cur_date if ('g_cur_date' in dir() ) else time.strftime("%Y%m%d") )

print(log_file)

../logs/similarity_20221001w.log


In [ ]:
conn = sqlite3.connect('./worth1.db')
df_bal = pd.read_sql(' select * from balance ', conn) 
# rt_df.to_sql('balance', con=conn, if_exists='append', index=False) #,
conn.close()

conn = sqlite3.connect('../pkl/binance.db')
df_acc0 = pd.read_sql(' select * from account ', conn) 
# rt_df.to_sql('account', con=conn, if_exists='append', index=False) #,
df_bal0 = pd.read_sql(' select * from balance ', conn) 
df_ord0 = pd.read_sql(' select * from orders ', conn) 
conn.close()

print(df_acc0.shape)

# df_account0.to_pickle('%s/%s.pkl' % ('../pkl', "accountdf" ) )
df_account0 = pd.read_pickle('%s/%s.pkl' % ('../pkl', "accountdf" ) )

# df_balance0.to_pickle('%s/%s.pkl' % ('../pkl', "balancedf" ) )
df_balance0 = pd.read_pickle('%s/%s.pkl' % ('../pkl', "balancedf" ) )
#
# display(df_bal)


In [2]:
from gevent import pywsgi

In [3]:
# print(f"{df_bal} \n")

# print(f"{df_bal0} \n")

# print(f"{df_account0} \n")

# print(f"{df_balance0} \n")

# print(f"{df_acc0.loc[len(df_acc0.index)-1] } \n")

df_acc0.loc[len(df_acc0.index)] = ['bi_similarity_cb68', 'USDT', 2891.300537, 63.027725, 2980.668457, 152.3955763, 30.479115, 1, 3043.696045, 2891.300537, 0, 152.395569, '2022-09-2700:00:02', 63.027725, '2022-09-27 00:00:00.254', 1664208000254, 2980.668457]
df_acc0.loc[len(df_acc0.index)] = ['bi_similarity_cb68', 'USDT',  71.700699, 52.454514, 3059.062256, 2987.388388, 29.873884, 1, 3111.516602, 124.128311, 0, 2987.388428, '2022-09-27 07:01:25', 52.454514, '2022-09-27 07:01:25.570', 1664233285570, 3059.062256]
df_acc0.loc[len(df_acc0.index)] = ['bi_similarity_cb68', 'USDT',  2983.514893, 0, 2983.514893, 0, 0, 1, 2983.514893, 2983.514893, 0, 0, '2022-09-28 07:00:03', 0, '2022-09-28 00:00:03.060', 1664294403060, 2983.514893]


df_acc0['balance'] = df_acc0['walletBalance'] 
df_account0['balance'] = df_account0['walletBalance'] 
# print(f"{df_acc0[df_acc0['access']=='bi_similarity_cb68'  ] }")

dfb = pd.concat([df_bal, df_bal0, df_balance0, df_acc0, df_account0, ],  ignore_index=True) # join='inner',

# print(f"{dfb} \n")
display(dfb)

,accountAlias,asset,availableBalance,balance,crossUnPnl,crossWalletBalance,marginAvailable,maxWithdrawAmount,queryTime,updateDate,updateTime,access,initialMargin,maintMargin,marginBalance,openOrderInitialMargin,positionInitialMargin,unrealizedProfit,walletBalance
0,FzFzXqfWSgSgoC,USDT,106.127319,3059.560791,227.095184,3059.560791,1,106.127319,2022-09-28 16:37:08,2022-09-28 16:00:00.221000,1664352000221,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,FzuXfWTioCmYmY,USDT,4169.830566,4169.830566,0.000000,4169.830566,1,4169.830566,2022-09-26 19:00:05,2022-09-26 17:15:18.300000,1664183718300,bi_similarity_4f58,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,FzuXfWTioCmYmY,USDT,4169.830566,4169.830566,0.000000,4169.830566,1,4169.830566,2022-09-26 19:03:05,2022-09-26 17:15:18.300000,1664183718300,bi_similarity_4f58,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,FzuXfWTioCmYmY,USDT,4169.830566,4169.830566,0.000000,4169.830566,1,4169.830566,2022-09-26 19:03:51,2022-09-26 17:15:18.300000,1664183718300,bi_similarity_4f58,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,FzuXfWTioCmYmY,USDT,4169.830566,4169.830566,0.000000,4169.830566,1,4169.830566,2022-09-26 19:08:19,2022-09-26 17:15:18.300000,1664183718300,bi_similarity_4f58,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,NaN,USDT,2891.300537,2980.668457,63.027725,2980.668457,1,2891.300537,2022-09-2700:00:02,2022-09-27 00:00:00.254,1664208000254,bi_similarity_cb68,152.395576,30.479115,3043.696045,0.0,152.395569,63.027725,2980.668457
66,NaN,USDT,71.700699,3059.062256,52.454514,3059.062256,1,124.128311,2022-09-27 07:01:25,2022-09-27 07:01:25.570,1664233285570,bi_similarity_cb68,2987.388388,29.873884,3111.516602,0.0,2987.388428,52.454514,3059.062256
67,NaN,USDT,2983.514893,2983.514893,0.000000,2983.514893,1,2983.514893,2022-09-28 07:00:03,2022-09-28 00:00:03.060,1664294403060,bi_similarity_cb68,0,0.000000,2983.514893,0.0,0.000000,0.000000,2983.514893
68,NaN,USDT,2983.514893,2983.514893,0.000000,2983.514893,1,2983.514893,2022-09-28 07:00:03,2022-09-28 00:00:03.060000,1664294403060,bi_similarity_Test2022_cb68,0.00000000,0.000000,2983.514893,0.0,0.000000,0.000000,2983.514893


In [17]:
dfb1 = dfb.iloc[:,0:12]
dfb1['access'] = dfb1['access'].fillna('')
dfb2 = dfb1[ ~dfb1['access'].str.contains('4f58')  ].sort_values(by="updateTime").drop_duplicates(subset=['updateTime'], keep='last', inplace=False, ignore_index=False)

dfb2['accountAlias'] = 'FzFzXqfWSgSgoC'
dfb2['access'] = 'similarity'  # name  access
# dfb2.drop(['access'], axis=1, inplace=True)

# dfb2.columns = ['tradeDate', 'openTime', 'closeTime', 'closeSec', 'open', 'close']

dfb2.sort_index(axis=1, ascending=True, inplace=True)
dfb2.reset_index(drop=True,inplace=True)

display( dfb2 )

dfb2.to_pickle('%s/%s.pkl' % ('.', "worthdf" ) )
# df_balance0 = pd.read_pickle('%s/%s.pkl' % ('.', "worthdf" ) )


,access,accountAlias,asset,availableBalance,balance,crossUnPnl,crossWalletBalance,marginAvailable,maxWithdrawAmount,queryTime,updateDate,updateTime
0,similarity,FzFzXqfWSgSgoC,USDT,3000.000000,3000.000000,0.000000,3000.000000,1,3000.000000,2022-09-26 08:06:02,2022-09-25 23:42:53.136000,1664120573136
1,similarity,FzFzXqfWSgSgoC,USDT,2850.048828,2998.806152,0.898237,2998.806152,1,2850.048828,2022-09-26 08:07:00,2022-09-26 08:07:00.741000,1664150820741
2,similarity,FzFzXqfWSgSgoC,USDT,2891.300537,2980.668457,63.027725,2980.668457,1,2891.300537,2022-09-2700:00:02,2022-09-27 00:00:00.254,1664208000254
3,similarity,FzFzXqfWSgSgoC,USDT,71.700699,3059.062256,52.454514,3059.062256,1,124.128311,2022-09-27 07:01:25,2022-09-27 07:01:25.570,1664233285570
4,similarity,FzFzXqfWSgSgoC,USDT,2983.514893,2983.514893,0.000000,2983.514893,1,2983.514893,2022-09-28 07:00:03,2022-09-28 00:00:03.060000,1664294403060
5,similarity,FzFzXqfWSgSgoC,USDT,0.000000,2977.555664,4.452701,2977.555664,1,0.000000,2022-09-28 07:01:27,2022-09-28 07:01:27.599000,1664319687599
6,similarity,FzFzXqfWSgSgoC,USDT,106.127319,3059.560791,227.095184,3059.560791,1,106.127319,2022-09-28 16:37:08,2022-09-28 16:00:00.221000,1664352000221


In [16]:
# dfb2.to_pickle('%s/%s.pkl' % ('.', "worthdf" ) )
df_worth = pd.read_pickle('%s/%s.pkl' % ('.', "worthdf" ) )
print(f"{ df_worth.shape }")
print(f"{ df_worth['balance'][0] } , {df_worth['balance'][0] }")

print(f"{ df_worth.dtypes }")
display(df_worth)

(7, 12)
3000.0 , 3000.0
access                 object
accountAlias           object
asset                  object
availableBalance      float64
balance               float64
crossUnPnl            float64
crossWalletBalance    float64
marginAvailable         int64
maxWithdrawAmount     float64
queryTime              object
updateDate             object
updateTime              int64
dtype: object


,access,accountAlias,asset,availableBalance,balance,crossUnPnl,crossWalletBalance,marginAvailable,maxWithdrawAmount,queryTime,updateDate,updateTime
0,similarity,FzFzXqfWSgSgoC,USDT,3000.000000,3000.000000,0.000000,3000.000000,1,3000.000000,2022-09-26 08:06:02,2022-09-25 23:42:53.136000,1664120573136
1,similarity,FzFzXqfWSgSgoC,USDT,2850.048828,2998.806152,0.898237,2998.806152,1,2850.048828,2022-09-26 08:07:00,2022-09-26 08:07:00.741000,1664150820741
2,similarity,FzFzXqfWSgSgoC,USDT,2891.300537,2980.668457,63.027725,2980.668457,1,2891.300537,2022-09-2700:00:02,2022-09-27 00:00:00.254,1664208000254
3,similarity,FzFzXqfWSgSgoC,USDT,71.700699,3059.062256,52.454514,3059.062256,1,124.128311,2022-09-27 07:01:25,2022-09-27 07:01:25.570,1664233285570
4,similarity,FzFzXqfWSgSgoC,USDT,2983.514893,2983.514893,0.000000,2983.514893,1,2983.514893,2022-09-28 07:00:03,2022-09-28 00:00:03.060000,1664294403060
5,similarity,FzFzXqfWSgSgoC,USDT,0.000000,2977.555664,4.452701,2977.555664,1,0.000000,2022-09-28 07:01:27,2022-09-28 07:01:27.599000,1664319687599
6,similarity,FzFzXqfWSgSgoC,USDT,106.127319,3059.560791,227.095184,3059.560791,1,106.127319,2022-09-28 16:37:08,2022-09-28 16:00:00.221000,1664352000221


In [20]:

df_worth['queryTime'] = df_worth['queryTime'].astype('str')

conn = sqlite3.connect('./worth.db')
# df_bal = pd.read_sql(' select * from balance ', conn) 
df_worth.to_sql('balance', con=conn, if_exists='append', index=False) #,
conn.close()

In [25]:
# conn = sqlite3.connect('./worth.db')
conn = sqlite3.connect('../pkl/binance.db')
orders_df = pd.read_sql(' select * from orders  ', conn)
conn.close()

In [26]:
# print(f"{orders_df}")
# display(orders_df)

conn = sqlite3.connect('./worth.db')
# df_ord = pd.read_sql(' select * from orders ', conn) 
orders_df.to_sql('orders', con=conn, if_exists='append', index=False) #,
conn.close()

In [27]:
display(orders_df)

,access,avgPrice,clientOrderId,closePosition,cumQuote,executedQty,orderId,origQty,origType,positionSide,price,priceProtect,reduceOnly,side,status,stopPrice,symbol,time,timeInForce,type,updateDate,updateTime,workingType
0,bi_similarity_Test2022_cb68,0.00000,SIB20220930083425MKTbtcusdtGTC0C0A1B,0,0,0,3220729389,0.068,MARKET,BOTH,0,0,0,BUY,NEW,0,BTCUSDT,2022-09-30 08:34:25,GTC,MARKET,2022-09-30 08:34:25.480000,1664498065480,CONTRACT_PRICE
1,bi_similarity_at_4f58,0.00000,SIB20220930095226MKTbtcusdtGTC0C0A1B,0,0,0,3220897197,0.209,MARKET,BOTH,0,0,0,BUY,NEW,0,BTCUSDT,2022-09-30 09:52:26,GTC,MARKET,2022-09-30 09:52:26.424000,1664502746424,CONTRACT_PRICE
2,bi_similarity_at_4f58,0.00000,SIS20220930160702MKTbtcusdtGTC0C0A1B,0,0,0,3221544187,0.209,MARKET,BOTH,0,0,0,SELL,NEW,0,BTCUSDT,2022-09-30 16:07:02,GTC,MARKET,2022-09-30 16:07:02.667000,1664525222667,CONTRACT_PRICE
3,bi_similarity_at_4f58,0.00000,SIB20220930171624MKTbtcusdtGTC0C0A1B,0,0,0,3221595604,0.207,MARKET,BOTH,0,0,0,BUY,NEW,0,BTCUSDT,2022-09-30 17:16:24,GTC,MARKET,2022-09-30 17:16:24.828000,1664529384828,CONTRACT_PRICE
4,bi_similarity_at_4f58,0.00000,SIS20220930172002MKTbtcusdtGTC0C0A1B,0,0,0,3221598191,0.207,MARKET,BOTH,0,0,0,SELL,NEW,0,BTCUSDT,2022-09-30 17:20:02,GTC,MARKET,2022-09-30 17:20:02.114000,1664529602114,CONTRACT_PRICE
5,bi_similarity_at_4f58,0.00000,SIB20220930172324MKTbtcusdtGTC0C0A1B,0,0,0,3221600485,0.203,MARKET,BOTH,0,0,0,BUY,NEW,0,BTCUSDT,2022-09-30 17:23:24,GTC,MARKET,2022-09-30 17:23:24.855000,1664529804855,CONTRACT_PRICE
6,bi_similarity_at_4f58,0.00000,SIS20220930173902MKTbtcusdtGTC0C0A1B,0,0,0,3221610806,0.203,MARKET,BOTH,0,0,0,SELL,NEW,0,BTCUSDT,2022-09-30 17:39:02,GTC,MARKET,2022-09-30 17:39:02.136000,1664530742136,CONTRACT_PRICE
7,bi_similarity_Test2022_cb68,0.00000,SIS20221001000501MKTbtcusdtGTC0C0A1B,0,0,0,3222389708,0.185,MARKET,BOTH,0,0,0,SELL,NEW,0,BTCUSDT,2022-10-01 00:05:01,GTC,MARKET,2022-10-01 00:05:01.956000,1664553901956,CONTRACT_PRICE


In [3]:
# conn = sqlite3.connect('./worth.db')
conn = sqlite3.connect('../pkl/binance.db')
factor_df = pd.read_sql(' select * from factor  ', conn)
conn.close()

In [4]:
display(factor_df)

,name,updateDateTime,clientOrderId,status,symbol,side,dirction,total,orderPrice,quantity,orderType,factor,updateTime,note
0,similarity,2022-09-30 17:20:02,SIS20220930172002MKTbtcusdtGTC0C0A1B,similarity,BTCUSDT,SELL,Close,2027.208984,19498.0,0.207,MARKET,"corrDate:, corrVal:",1664529602167,--
1,similarity,2022-09-30 17:23:24,SIB20220930172324MKTbtcusdtGTC0C0A1B,similarity,BTCUSDT,BUY,Open,3966.145508,19464.6,0.203,MARKET,"corrDate:20220724, corrVal:0.6053420059880656",1664529804909,--
2,similarity,2022-09-30 17:39:02,SIS20220930173902MKTbtcusdtGTC0C0A1B,similarity,BTCUSDT,SELL,Close,1978.569092,19450.1,0.203,MARKET,"corrDate:, corrVal:",1664530742192,--
3,similarity,2022-10-01 00:05:02,SIS20221001000501MKTbtcusdtGTC0C0A1B,similarity,BTCUSDT,SELL,Close,971.515625,19666.0,0.185,MARKET,"corrDate:, corrVal:",1664553902012,--
4,similarity,2022-10-01 07:01:26,SIB20221001070125MKTbtcusdtGTC0C0A1B,similarity,BTCUSDT,BUY,Open,2789.692627,19368.7,0.144,MARKET,"corrDate:20220713, corrVal:0.8415807516482112",1664578886091,--


In [6]:
conn2 = sqlite3.connect('../pkl/binance.db')
factor_df = pd.read_sql(' select * from factor  ', conn2)
conn2.close()
tradeCount = factor_df.iloc[0,0]

# print(tradeCount, factor_df)
display()

similarity          name       updateDateTime                         clientOrderId      status   symbol  side dirction        total  orderPrice quantity orderType                                         factor     updateTime note
0  similarity  2022-09-30 17:20:02  SIS20220930172002MKTbtcusdtGTC0C0A1B  similarity  BTCUSDT  SELL    Close  2027.208984     19498.0    0.207    MARKET                            corrDate:, corrVal:  1664529602167   --
1  similarity  2022-09-30 17:23:24  SIB20220930172324MKTbtcusdtGTC0C0A1B  similarity  BTCUSDT   BUY     Open  3966.145508     19464.6    0.203    MARKET  corrDate:20220724, corrVal:0.6053420059880656  1664529804909   --
2  similarity  2022-09-30 17:39:02  SIS20220930173902MKTbtcusdtGTC0C0A1B  similarity  BTCUSDT  SELL    Close  1978.569092     19450.1    0.203    MARKET                            corrDate:, corrVal:  1664530742192   --
3  similarity  2022-10-01 00:05:02  SIS20221001000501MKTbtcusdtGTC0C0A1B  similarity  BTCUSDT  SELL    Close 